In [14]:
import os
import MySQLdb
from dotenv import load_dotenv
import pandas as pd
import numpy as np
load_dotenv()

def database_query(sql_query):
    try:
        connection = MySQLdb.connect(
        host=os.getenv("DATABASE_HOST"),
        user=os.getenv("DATABASE_USERNAME"),
        passwd=os.getenv("DATABASE_PASSWORD"),
        db=os.getenv("DATABASE"),
        autocommit=True,
        # ssl_mode="VERIFY_iDENTITY",
        ssl={"ca": "/etc/ssl/certs/ca-certificates.crt"})
    except:
        connection = MySQLdb.connect(
        host=os.environ["DATABASE_HOST"],
        user=os.environ["DATABASE_USERNAME"],
        passwd=os.environ["DATABASE_PASSWORD"],
        db=os.environ["DATABASE"],
        autocommit=True,
        # ssl_mode="VERIFY_iDENTITY",
        ssl={"ca": "/etc/ssl/certs/ca-certificates.crt"})
    try:
        c = connection.cursor()
        c.execute(sql_query)
        results = c.fetchall()
        return results
    except MySQLdb.Error as e:
        print("MySQL Error:", e)
    finally:
        c.close()
        connection.close()


sql_query = '''

select
    title
    ,department
    ,location
    ,salary
    ,closing_date
    ,uid
    ,scraped_date
from 
    all_time_listings

'''

df = pd.DataFrame(database_query(sql_query))
df.columns = ['title', 'department', 'location', 'salary', 'closing_date', 'uid', 'scraped_date']
df.head()

,title,department,location,salary,closing_date,uid,scraped_date
0,Senior Data Analyst - Corporate Performance,Companies House,"Cardiff, Wales, CF14 3UZ","46,588",2024-03-05,153376,2024-02-21
1,Kitchen Steward,House of Commons,Westminster,"24,959",2024-05-19,200324,2024-01-04
2,National Professional Advisor (for People with...,Care Quality Commission,"East Midlands (England), East of England, Lond...","70,000",2024-02-26,24711,2024-02-13
3,Commis Chef,House of Commons,Westminster,"24,959",2024-05-19,255755,2024-01-04
4,Recreation Works Supervisor,Forestry Commission,"Burley Office, Burley, Hampshire BH24 4HS","26,534",2024-01-14,257501,2024-01-04


In [15]:
from plotnine import ggplot, scale_fill_gradient, scale_color_continuous, scale_color_gradient, aes, geom_point, geom_col, geom_line, geom_histogram, geom_boxplot, facet_wrap, theme, element_text, element_blank, element_rect, element_line, labs, scale_x_continuous, scale_y_continuous, scale_fill_manual, scale_color_manual, scale_linetype_manual, scale_shape_manual, scale_size_manual, scale_alpha_manual, coord_flip, coord_cartesian, coord_fixed

Plot showing median salaries by department

In [16]:
df['salary_int'] = df['salary'].str.replace(',','')
df['salary_int'] = df['salary_int'].astype(float)
df['scraped_date_date'] = pd.to_datetime(df['scraped_date'])
df['week_commencing'] = df['scraped_date_date'].dt.to_period('W').dt.start_time
df['salary_int'].describe()
daily_salary = df[['week_commencing','salary_int','department','location']].groupby(['week_commencing','salary_int','department','location']).mean().reset_index()
department_salary = df.groupby('department').agg({'salary_int':'median','uid':'count'}).reset_index().sort_values('salary_int', ascending=False)
department_salary.columns = ['department','salary_int','Number of Postings']

# department_salary = pd.concat([department_salary.nlargest(5, 'salary_int').reset_index(),department_salary.nsmallest(5, 'salary_int').reset_index()], axis=0, ignore_index=True).sort_values('salary_int', ascending=False)

plot = (
    ggplot(department_salary, aes(x="reorder(department, salary_int, ascending=False)", y='salary_int'
                                    ,color='Number of Postings', fill='Number of Postings'
                                   ))

    + geom_col()
    # + theme(legend_position='none')
    + scale_color_gradient(limits=[10, 100], labels=['<10', '>100'], breaks=[10, 100])
    + scale_fill_gradient(limits=[10, 100], labels=['<10', '>100'], breaks=[10, 100])
    + labs(title=f"Civil Service Departments Median Salary Comparison: {df['scraped_date_date'].dt.date.min()} to {df['scraped_date_date'].dt.date.max()}"
           , x='Department'
           , y='Median Salary (£)'
           , caption='Source: Civil Service Jobs (https://www.civilservicejobs.service.gov.uk/)')
    + theme(axis_text_x=element_text(angle=90, face='bold'))
    + theme(axis_text_x=element_text(size=6))
    + theme(axis_text_y=element_text(size=6))
    + theme(plot_caption=element_text(size=6, face='italic'))
    + theme(plot_title=element_text(size=20, face='bold'))
)

plot.save("department_salary.png", width=20, height=10, units='in', dpi=300)


/home/codespace/.python/current/lib/python3.10/site-packages/plotnine/ggplot.py:604: PlotnineWarning: Saving 20 x 10 in image.
/home/codespace/.python/current/lib/python3.10/site-packages/plotnine/ggplot.py:605: PlotnineWarning: Filename: department_salary.png


In [17]:
##department median salary QA

pass_qa = True
number_of_departments = 0
def qa_check(df, department):
    global pass_qa
    global number_of_departments
    department_median = df[df['department']==department]['salary_int'].values[0]
    df_department = df[df['department']==department]
    difference = department_median - df_department['salary_int'].describe().values[5]
    if difference > 0:
        print(f"Variation in QA check for {department}")
        pass_qa = False
    else:
        number_of_departments += 1


for i in df['department'].unique():
    qa_check(department_salary, i)
if pass_qa:
    print("QA checks passed. Number of departments checked: ", number_of_departments)
else:
    print("QA checks failed")

QA checks passed. Number of departments checked:  161


See if we can pull number of posts out of the data as well to improve salary averages?
Compare highest salaries with behvaiours to establish highest-value behaviours

In [18]:
application_df = pd.DataFrame(database_query('select * from all_time_ad_qualities'))

In [19]:
application_df.columns = [
        'uid', 
        'developing_self_and_others', 
        'leadership',
        'making_effective_decisions',
        'seeing_the_big_picture',
        'managing_a_quality_service',
        'working_together',
        'communicating_and_influencing',
        'changing_and_improving',
        'delivering_at_pace',
        'apply_at_advertisers_site',
        'cv',
        'personal_statement',
        'reference_request',
        'application_form',
        'cover_letter',
        'presentation',
        'interview',
        'portfolio',
        'test']

In [20]:
df['uid'] = df['uid'].astype(int)
application_df['uid'] = application_df['uid'].astype(int)
combined_df = pd.merge(df, application_df, on='uid', how='left')
combined_df.head(1)

,title,department,location,salary,closing_date,uid,scraped_date,salary_int,scraped_date_date,week_commencing,...,apply_at_advertisers_site,cv,personal_statement,reference_request,application_form,cover_letter,presentation,interview,portfolio,test
0,Senior Data Analyst - Corporate Performance,Companies House,"Cardiff, Wales, CF14 3UZ","46,588",2024-03-05,153376,2024-02-21,46588.0,2024-02-21,2024-02-19,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [21]:
behaviours_df = combined_df[['department', 'week_commencing', 'salary_int',
        'developing_self_and_others', 
        'leadership',
        'making_effective_decisions',
        'seeing_the_big_picture',
        'managing_a_quality_service',
        'working_together',
        'communicating_and_influencing',
        'changing_and_improving',
        'delivering_at_pace']]

def salary_bucket_func(salary_int):
        bucket = '£90k+'
        salary_bucket_dict = {
                90000: '£70k-£90k'
                , 70000: '£50k-£70k'
                , 50000: '£30k-£50k'
                , 30000: '<£30k'}
        try:
                for salary in salary_bucket_dict.keys():
                        if salary_int <= salary:
                                bucket = salary_bucket_dict[salary]
                                

                
        except:
                bucket = 'bad entry'
        return bucket

salary_bucket_func_vectorised = np.vectorize(salary_bucket_func)
behaviours_df['salary_bucket'] = salary_bucket_func_vectorised(behaviours_df['salary_int'])

grouped_behaviours = behaviours_df[['department'
                                    , 'week_commencing'
                                    , 'salary_bucket'
                                    , 'developing_self_and_others'
                                    , 'leadership'
                                    , 'making_effective_decisions'
                                    , 'seeing_the_big_picture'
                                    , 'managing_a_quality_service'
                                    , 'working_together'
                                    , 'communicating_and_influencing'
                                    , 'changing_and_improving'
                                    , 'delivering_at_pace']].groupby(['department','week_commencing','salary_bucket']).sum().reset_index()

/tmp/ipykernel_2597/519122141.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
melted_behaviours = pd.melt(grouped_behaviours, id_vars=['department', 'week_commencing', 'salary_bucket'], value_vars=['developing_self_and_others', 'leadership', 'making_effective_decisions', 'seeing_the_big_picture', 'managing_a_quality_service', 'working_together', 'communicating_and_influencing', 'changing_and_improving', 'delivering_at_pace'])
melted_behaviours.columns = ['department', 'week_commencing', 'salary_bucket', 'behaviour', 'count']

melted_behaviours = melted_behaviours[['salary_bucket', 'behaviour', 'count']].groupby(['salary_bucket', 'behaviour']).sum().reset_index()

def behaviour_cleaner(behaviour):
    behaviour_dict = {
        'developing_self_and_others': 'Developing Self and Others'
        , 'leadership': 'Leadership'
        , 'making_effective_decisions': 'Making Effective Decisions'
        , 'seeing_the_big_picture': 'Seeing the Big Picture'
        , 'managing_a_quality_service': 'Managing a Quality Service'
        , 'working_together': 'Working Together'
        , 'communicating_and_influencing': 'Communicating and Influencing'
        , 'changing_and_improving': 'Changing and Improving'
        , 'delivering_at_pace': 'Delivering at Pace'
    }
    return behaviour_dict[behaviour]

melted_behaviours['behaviour'] = melted_behaviours['behaviour'].apply(behaviour_cleaner)

df['salary_bucket'] = salary_bucket_func_vectorised(df['salary_int'])
df = df[df['salary_bucket']!='bad entry']
total_bucket_counts = df['salary_bucket'].value_counts().reset_index()
total_bucket_counts.columns = ['salary_bucket', 'total_count']
melted_behaviours['cum_sum'] = (melted_behaviours.groupby(['salary_bucket'])['count'].transform('max'))
melted_behaviours = pd.merge(melted_behaviours, total_bucket_counts, how='left')
melted_behaviours['percentage'] = (melted_behaviours['count'] / melted_behaviours['total_count']) * 100

behaviour_plot = (
    ggplot(melted_behaviours, aes(x='behaviour', y='percentage', color='behaviour', fill='behaviour'))
    + geom_col()
    + facet_wrap('salary_bucket', nrow=1)
    + labs(title='Behaviours Requested by Salary Bucket'
           , x=''
           , y='Percentage of Postings Mentioning Behaviour (%)'
           , caption='Source: Civil Service Jobs (https://www.civilservicejobs.service.gov.uk/)')
    + theme(axis_title_y=element_text(size='14'))
    # + theme(legend_position='none')
    + theme(axis_text_x=element_text(color='white'))
    + theme(axis_ticks_x=element_text(color='white'))
    + theme(legend_title=element_blank())
    #custom fill colours
    + scale_fill_manual(values=['#87CEEB', '#FFD300', '#FC6A03', '#FFC0CB', '#7E38B7', '#378805', '#1E2F97', '#7c3f00', '#2f2f2f'])
    + scale_color_manual(values=['#87CEEB', '#FFD300', '#FC6A03', '#FFC0CB', '#7E38B7', '#378805', '#1E2F97', '#7c3f00', '#2f2f2f'])
)

behaviour_plot.save("behaviour_plot.png", width=20, height=10, units='in', dpi=300)

/home/codespace/.python/current/lib/python3.10/site-packages/plotnine/ggplot.py:604: PlotnineWarning: Saving 20 x 10 in image.
/home/codespace/.python/current/lib/python3.10/site-packages/plotnine/ggplot.py:605: PlotnineWarning: Filename: behaviour_plot.png


In [23]:
behaviour_qa_pass = True
behaviours = melted_behaviours['behaviour'].unique()
bad_uids = []
def behaviour_qa(input_df, behaviour, melted_df, chart_df):
    global behaviour_qa_pass
    global bad_uids


    behaviour_swap_back  = {
        'Developing Self and Others': 'developing_self_and_others'
        , 'Leadership': 'leadership'
        , 'Making Effective Decisions': 'making_effective_decisions'
        , 'Seeing the Big Picture': 'seeing_the_big_picture'
        , 'Managing a Quality Service': 'managing_a_quality_service'
        , 'Working Together': 'working_together'
        , 'Communicating and Influencing': 'communicating_and_influencing'
        , 'Changing and Improving': 'changing_and_improving'
        , 'Delivering at Pace': 'delivering_at_pace'
    }
    lower_behaviour = behaviour_swap_back[behaviour]    
    melted_df = melted_df[melted_df[lower_behaviour]==1][['uid', lower_behaviour]].sort_values('uid').reset_index(drop=True)
    

    total_found = input_df[lower_behaviour].sum()

    input_beheaviour_uids = input_df[input_df[lower_behaviour]==1]
    input_beheaviour_uids = input_beheaviour_uids[['uid', lower_behaviour]].sort_values('uid').reset_index(drop=True)
    melted_total_found = melted_df[lower_behaviour].sum()
    melted_df[lower_behaviour] = melted_df[lower_behaviour].astype(int)
    processed_uids = melted_df['uid'].values
    original_uids = input_beheaviour_uids['uid'].values
    for i in original_uids:
        if i not in processed_uids:
            # print(f"QA check failed for {behaviour}, {i} not found in melted dataframe")
            bad_uids.append(i)
            behaviour_qa_pass = False
    

    chart_df = chart_df[chart_df['behaviour']==behaviour]
    chart_df_total = chart_df['count'].sum()
    if total_found != chart_df_total:
        print(f"QA check failed for {behaviour}, total found in input dataframe: {total_found}, total found in melted dataframe: {melted_total_found}")
        behaviour_qa_pass = False
    else:
        behaviour_qa_pass = True
        print(f"QA check passed for {behaviour}, total found in input dataframe: {total_found}, total found in melted dataframe: {melted_total_found}")
    



for behaviour in behaviours:
    behaviour_qa(application_df, behaviour, combined_df, melted_behaviours)    
if behaviour_qa_pass == False:
    print("QA checks failed, these are the mismatched uids: ", bad_uids)


QA check passed for Changing and Improving, total found in input dataframe: 2264, total found in melted dataframe: 2264.0
QA check passed for Communicating and Influencing, total found in input dataframe: 2449, total found in melted dataframe: 2449.0
QA check passed for Delivering at Pace, total found in input dataframe: 2682, total found in melted dataframe: 2682.0
QA check failed for Developing Self and Others, total found in input dataframe: 2134, total found in melted dataframe: 2132.0
QA check passed for Leadership, total found in input dataframe: 2696, total found in melted dataframe: 2696.0
QA check passed for Making Effective Decisions, total found in input dataframe: 2108, total found in melted dataframe: 2108.0
QA check passed for Managing a Quality Service, total found in input dataframe: 2395, total found in melted dataframe: 2395.0
QA check passed for Seeing the Big Picture, total found in input dataframe: 2223, total found in melted dataframe: 2223.0
QA check passed for W

In [24]:
#need to debug this function to return uids where df['title'] contains any of the words in the list; won't take long but will need to pick up later 

def uids_filtered_on_title(df, title: list):
    uids = []
    df['title'] = df['title'].str.lower()
    for i in title:
        uids += df[df['title'].str.contains(i)]['uid'].values.tolist()
    return set(uids)

analyst_uids = uids_filtered_on_title(df, ['data', 'analy'])

def uids_filtered_on_salary_int(df, salary_range: list):
    return df[df['salary_int']>=salary_range[0] and df['salary_int'] <= salary_range[1]]['uid']

g7_g6_uids = uids_filtered_on_salary_int(df, [50000, 70000])

analyst_application_df = application_df[application_df['uid'].isin(analyst_uids)]
g7_analyst_application_df = application_df[application_df['uid'].isin(g7_g6_uids) & application_df['uid'].isin(analyst_uids)]

analyst_application_df['post_type'] = 'analyst_posts'
g7_analyst_application_df['post_type'] = 'g7_analyst_posts'
application_df['post_type'] = 'all_posts'

def application_process_prep(application_df, posting_type):

    application_process_df = application_df[[
            'uid', 
            'apply_at_advertisers_site',
            'cv',
            'personal_statement',
            'reference_request',
            'application_form',
            'cover_letter',
            'presentation',
            'interview',
            'portfolio',
            'test']]

    melted_application_process_df = pd.melt(application_process_df, id_vars=['uid'], value_vars=['apply_at_advertisers_site', 'cv', 'personal_statement', 'reference_request', 'application_form', 'cover_letter', 'presentation', 'interview', 'portfolio', 'test'])
    melted_application_process_df.columns = ['uid', 'application_process', 'appearances']
    melted_application_process_df = melted_application_process_df[['application_process', 'appearances']]
    melted_application_process_df = melted_application_process_df.groupby('application_process').sum().reset_index()

    def application_process_cleaner(application_process):
        application_process_dict = {
            'apply_at_advertisers_site': 'Apply at Advertisers Site'
            , 'cv': 'CV'
            , 'personal_statement': 'Personal Statement'
            , 'reference_request': 'Reference Request'
            , 'application_form': 'Application Form'
            , 'cover_letter': 'Cover Letter'
            , 'presentation': 'Presentation'
            , 'interview': 'Interview'
            , 'portfolio': 'Portfolio'
            , 'test': 'Online Test'
        }
        return application_process_dict[application_process]

    melted_application_process_df['Application Process'] = melted_application_process_df['application_process'].apply(application_process_cleaner)
    melted_application_process_df['posting_type'] = posting_type
    return melted_application_process_df

from plotnine import geom_bar

analyst_application_process_df = application_process_prep(analyst_application_df, 'analyst_posts')
g7_analyst_application_process_df = application_process_prep(g7_analyst_application_df, 'g7_analyst_posts')
all_posts_application_process_df = application_process_prep(application_df, 'all_posts')


combined_application_process_df = pd.concat([analyst_application_process_df, g7_analyst_application_process_df, all_posts_application_process_df], axis=0, ignore_index=True)

application_process_plot = (

    ggplot(combined_application_process_df, aes(y='appearances', x='Application Process', fill='Application Process'))
    + geom_col()
    + facet_wrap('posting_type', nrow=1)
    + labs(title='Most Common Application Elements'
           , y='Job Postings Mentioning Element'
           , caption='Source: Civil Service Jobs (https://www.civilservicejobs.service.gov.uk/)')
    + theme(axis_text_y=element_text(face='bold'))
    #remove x axis text
    + theme(axis_text_x=element_text(color='white'))
    + theme(axis_ticks_x=element_text(color='white'))
    + theme(axis_text_y=element_text(color='white'))
    + theme(axis_ticks_y=element_text(color='white'))
)

application_process_plot

AttributeError: 'Series' object has no attribute 'contains'